This tutorial illustrates how to create a Roman Flat Field reference file from scratch or from an existing example.
==========================================================

In [ ]:
import roman_datamodels.datamodels as rdd
import roman_datamodels.stnode as rds
import numpy as np
import asdf
from astropy.time import Time

All Roman Datamodels are built from particular types of objects. 
Using these is require as it supports the validation machinery 
that ensures that the particular type of file meets the specified requirements.
These requirements are defined in the schema files associated with each
data model. Normally, unless you are responsible for defining a new kind
of file, you will not need to look at the contents of the schema files.
However, if you construct the contents of such a data model that are
inconsistent with the schemas, you will get error messages when trying
to create or save the reference file. Consider this a good thing.

**Note: there are changes planned for the Flat Field Reference file content. The content of this tutorial will be updated when those changes are implemented**

The first step in building one from scratch is to construct it from the bottom up. For the flat field there are a few things to construct. All the data models have a meta item. We start by creating it as Python dictionary that we will populate with the needed information. The creation below begins with items that are always the same for a flat field.

In [ ]:
meta = {'telescope': 'ROMAN', 'reftype': 'FLAT'}
# Create the information about instrument configuration for this flat field.
# The first item is always the same, but the others depend on the particular flat field
instrument = {'name': 'WFI', 'detector': 'WFI01', 'optical_element': 'F129'}
meta['instrument'] = instrument
# All the rest of these must be provided by the person who has constructed the flat field
meta['pedigree'] = """whatever text you wish to describe the pedigree
    and it may go on and on and on and on and on and on
    and on and on and on and on and on
    and on and on, to your heart's content"""
meta['description'] = "whatever description you wish to provide"
meta['author'] = 'Wally Wombat'
meta['useafter'] = Time('2022-01-01T11:11:11.111') # the good old useafter date

Note that the pedigree and description values may be as long as you wish, book length if you so desire.

Now we will construct the main data model object.

In [ ]:
flat = rds.FlatRef()
flat['meta'] = meta
flat['data'] = np.ones((4096, 4096), dtype=np.float32) # You will supply a real flat, of course
flat['dq'] = np.zeros((4096, 4096), dtype=np.uint32) # likewise (this will soon be changed to uint16)
flat['err'] = np.zeros((4096, 4096), dtype=np.float32) # likewise

Now we will construct an ASDF file and save it using these contents.

In [ ]:
af = asdf.AsdfFile()
af.tree = {'roman': flat}
af.write_to('my_first_flat.asdf')

To see what happens when we don't follow the rules, we will try the same with a few illegal variants.

In [ ]:
flat.meta.telescope = 'HST'

This is because the relevant schema requires ROMAN

In [ ]:
flat.data = np.arange(100)

In [ ]:
af2 = asdf.AsdfFile()
af2.tree = {'roman': flat}
af2.write_to('bad_flat.asfd')

This error arises because the data array is the wrong numerical type and dimensionality.

OK, let's try opening the file we successfully made.

In [ ]:
flatdm = rdd.open('my_first_flat.asdf')

In [ ]:
flatdm.meta

In [ ]:
flatdm.meta.telescope

In [ ]:
flatdm.meta.instrument.optical_element

In [ ]:
print(flatdm.meta.pedigree)

That is about the essence of it. One can script this easily if that is 
easier, particularly if many files are sharing much info (such as author,
pedigree, description, etc).

Note that the attribute 'dot' notation can be used after the model is 
defined, but not when creating new attributes, where instead the
usual dictionary indexing notation must be used, as at the beginning
when creating from scratch.

The following illustrates that modifying an existing file and writing it out is even simpler.

In [ ]:
instrument['detector'] = 'WFI02'
instrument['optical_element'] = 'F062'
flatdm.meta.instrument = instrument
flatdm.save('my_second_flat.asdf')


There is currently a limitation on assigning model attributes more than
one layer lower than the schema they are defined in, but that will be
rectified in the near future.